In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('jugadores_ready.csv')

In [5]:
list_equip = list(df['equipo'].unique())

In [15]:
def Team(E):
    partidos = df.groupby('equipo').get_group(E)
    x = 1 #jornada a empezar
    lista = []
    for i in range(36): #numero de filas
        datos_a = pd.DataFrame(np.concatenate(np.asarray(partidos[partidos.jornada == i+x].iloc[0:12,0:7]))).T
        datos_a = datos_a.reset_index()
        corners_a = partidos[partidos.jornada == i+x].iloc[0:12,-1].sum()#suma de los corners
        datos_a.insert(6,'corners', corners_a)


        datos_b = pd.DataFrame(np.concatenate(np.asarray(partidos[partidos.jornada == i+x+1].iloc[0:12,0:7]))).T
        datos_b = datos_b.reset_index()
        corners_b = partidos[partidos.jornada == i+x+1].iloc[0:12,-1].sum()#suma de los corners
        datos_b.insert(6,'corners', corners_b)

        jugadores_c = pd.DataFrame(np.concatenate(np.asarray(partidos[partidos.jornada == i+x+2].iloc[0:12,0:1]))).T
        corners_c = partidos[partidos.jornada == i+x+2].iloc[0:12,-1].sum()
        jugadores_c.insert(11,'corners', corners_c)

        total_v1 = pd.concat([datos_a,datos_b,jugadores_c],axis=1,ignore_index=True)
        total_v1 = total_v1.drop([79,86,87,88,89,91],axis=1)
        lista.append(total_v1)

    df_final = pd.concat(lista)
    df_final = df_final.reset_index()

    df_final = df_final.drop(0,axis=1)
    df_final = df_final.drop('index',axis=1)
    
    
    return df_final

## Random Forest

In [16]:
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer
from sklearn import svm
from sklearn.model_selection import LeaveOneOut
from sklearn.neighbors import NearestCentroid
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [17]:
def SVMTeam(X):
    #df_final=X    
    df_final = Team(X)
    df_final = df_final.replace(np.nan, 0.0)
    df_final[169] = df_final[169].replace((0,1,2,3),0)
    df_final[169] = df_final[169].replace((4,5,6),1)
    df_final[169] = df_final[169].replace((7,8,9,10,11,12,13),2)
    X = df_final.iloc[:,:162]
    y = df_final.iloc[:,162:163]
    X = np.array(X)
    y = np.array(y)
    #y = y.replace(np.nan,0.0)
    y = y[:, 0]
    # Build a forest and compute the feature importances
    forest =  clf = svm.SVC(kernel='sigmoid')

 #Change for SVM
   # forest.fit(X, y)
    #importances = forest.feature_importances_
    #std = np.std([tree.feature_importances_ for tree in forest.estimators_], axis=0)
    #indices = np.argsort(importances)[::-1]

    # Print the feature ranking
    #print("Feature ranking:")

    #for f in range(X.shape[1]):
      #  print("%d. feature %d (%f)" % (f + 1, indices[f], importances[indices[f]]))
    #X = df_final.iloc[:,indices[:50]]
    #X=X.replace(np.nan,0)
    #X = np.array(X)

    loo = LeaveOneOut()
    loo.get_n_splits(X)
    yhat=[]
    for train_index, test_index in loo.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        #CLASIFICADOR SVM
        clf = svm.SVC(kernel='sigmoid')
        clf.fit(X_train, y_train)  ##X_train, y_train
        yhat1 = clf.predict(X_test)
        yhat.append(yhat1)       
        
    #print("Accuracy: ", accuracy_score(y,yhat))
    return accuracy_score(y,yhat)

In [18]:
L_Acc = []
for i in list_equip:
    L_Acc.append(SVMTeam(i))
Dicc = dict(zip(list_equip,L_Acc))
Dicc

{'girona': 0.6666666666666666,
 'valladolid': 0.4166666666666667,
 'betis': 0.5,
 'levante': 0.4166666666666667,
 'celta': 0.0,
 'espanyol': 0.5,
 'villarreal': 0.3055555555555556,
 'real_sociedad': 0.5277777777777778,
 'barcelona': 0.0,
 'alaves': 0.5833333333333334,
 'eibar': 0.4444444444444444,
 'huesca': 0.5,
 'rayo': 0.5,
 'sevilla': 0.5277777777777778,
 'real_madrid': 0.4166666666666667,
 'getafe': 0.5,
 'valencia': 0.4722222222222222,
 'atletico': 0.4166666666666667,
 'athletic': 0.6944444444444444,
 'leganes': 0.5277777777777778}

In [19]:
#Media Adaboost: 0.4375
Media_Random_Forest = sum(L_Acc)/20
Media_Random_Forest

0.4458333333333334

## KERNELS: linear=0.41, poly=0.43, rbf=0.425, sigmoid=0.44 (En sigmoid hay 3 que dan 0, si obviamos esos, la media es de 0.5)